In [1]:
# !pip install ollama
# !pip install langchain chromadb gradio 
# !pip install -U langchain-community
# !pip install pymupdf

In [1]:
import ollama
import gradio as gr
import shutil

from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
  
from langchain_community.embeddings import OllamaEmbeddings

import re
import os

In [13]:
def cleanup_vectorstore():
    global vectorstore, retriever
    try:
        if 'vectorstore' in globals():
            del vectorstore
        if 'retriever' in globals():
            del retriever
        print("✅ Variables nettoyées")
    except:
        pass

In [25]:
def process_all_pdfs(pdf_folder="./pdf"):
    # Nettoyage de la base de données vectorielle avant de la recréer
    cleanup_vectorstore()

    all_chunks = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

    for filename in os.listdir(pdf_folder):
        filepath = os.path.join(pdf_folder, filename)
        loader = PyMuPDFLoader(filepath)
        data = loader.load()
        chunks = text_splitter.split_documents(data)
        # Ajoute le titre du fichier dans les métadonnées
        for doc in chunks:
            doc.metadata["title_"] = filename
        all_chunks.extend(chunks)

    for chunk in all_chunks:
        metadata = (
            "[source: " + str(chunk.metadata.get("title_")) +
            " | author: " + str(chunk.metadata.get("author")) +
            " | creationdate: " + str(chunk.metadata.get("creationdate")) +
            " | page: " + str(chunk.metadata.get("page")) + "]"
        )
        chunk.page_content = metadata + " " + chunk.page_content
    
    # Transphorme texte en vecteurs
    embeddings = OllamaEmbeddings(model="bge-m3")
    # Crée la bd vectorielle
    vectorstore = Chroma.from_documents(documents=all_chunks, embedding=embeddings, persist_directory="./chroma_db")
    # Object pour la recherche
    retriever = vectorstore.as_retriever()

    print("✅ Base de données vectorielle créée avec succès !")
    return text_splitter, vectorstore, retriever

In [17]:
def combine_docs(docs):
    l_docs = []
    for doc in docs:
        if doc.page_content not in l_docs:
            l_docs.append(doc.page_content)
    return "\n\n".join(l_docs)

In [19]:
def ollama_llm(question, context):
    formatted_prompt = (
        f"Tu es un assistant expert qui répond en français uniquement.\n"
        f"Voici une question posée par un utilisateur, accompagnée d'un contexte issu de documents :\n\n"
        f"Contexte :\n{context}\n\n"
        f"Question : {question}\n\n"
        f"Fournis une réponse claire, précise et en français uniquement."
    )
    response = ollama.chat(
        model="gemma3n:e2b",
        messages=[{'role': 'user', 'content': formatted_prompt}]
    )

    response_content = response['message']['content']
    final_answer = re.sub(r'<think>.*?</think>',
                          '',
                          response_content,
                          flags=re.DOTALL).strip()
    return final_answer

# Fonctionnement sans interface

In [29]:
# Création de la base de données vectorielle
text_splitter, vectorstore, retriever = process_all_pdfs("./pdf")

✅ Variables nettoyées
✅ Base de données vectorielle créée avec succès !


In [31]:
question = "Quelle est la hauteur du platane ? Et sa largeur ?"
# Recherche dans la base de données vectorielle
docs = retriever.get_relevant_documents(question,k=4)
combined_text = combine_docs(docs)
# Exécution de la requête LLM
ollama_llm(question, combined_text)

'La hauteur du platané est de 35 mètres et sa largeur est de 20 mètres.'

# Avec interface

In [39]:
def ask_model(question):
    docs = retriever.get_relevant_documents(question, k=4)
    combined_text = combine_docs(docs)
    response = ollama_llm(question, combined_text)
    return response

In [40]:
# Création de l'interface
interface = gr.Interface(
    fn=ask_model,
    inputs=gr.Textbox(lines=2, label="Pose ta question"),
    outputs=gr.Textbox(label="Réponse du modèle"),
    title="Interface avec Ollama",
    description="Pose une question"
)
# Lancement
interface.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
